<a target="_blank" href="https://colab.research.google.com/github/chasubeen/Kwater/tree/subeen">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd

import os
import chardet

In [3]:
pd.set_option('display.float_format', None) # 과학적 표기법 x

# **1. 데이터 병합**

In [4]:
### 파일 병합을 위한 함수

def file_concat(path, file_list):
  dfs = []  # DataFrame들을 저장할 리스트

  for file in file_list:
    # 파일 확장자가 .csv인 경우에만 처리
    if file.endswith(".csv"):
      file_path = os.path.join(path, file)

      # 파일을 열어서 인코딩 확인
      with open(file_path, 'rb') as f:
        rawdata = f.read()
      result = chardet.detect(rawdata)
      enc = result['encoding']

      # 해당 파일의 인코딩에 맞게 열어서 DataFrame으로 변환
      df = pd.read_csv(file_path, encoding = enc)
      dfs.append(df)

  # DataFrame들을 세로 방향으로 이어붙임
  df = pd.concat(dfs, axis = 0, ignore_index = True)  # ignore_index = True로 해줘야 오류 없이 밑으로 결합됨

  return df

## **1-1. 저수량 데이터**
- 각 댐에 대한 정보를 포함하고 있는 데이터    
- 활용 데이터 목록
  - [다목적댐 운영 정보(일자료)](https://www.bigdata-environment.kr/user/data_market/detail.do?id=b77fd770-38bb-11ea-be28-4fa0eb812a46)
  - [MyWater 물정보포털수문자료](https://www.water.or.kr/kor/realtime/sumun/index.do?mode=sumun&menuId=13_91_93_94)

### **📌 Data Description**
1. dam_nm: 댐이름
2. obsr_de: 일자
3. rain_qy: 강우량
4. inflow_qy: 유입량
5. tot_dcwtr_qy: 방류량
6. rsvwt_qy: 저수량
7. dam_rsvwt_rt: 저수율

### **a) 데이터 불러오기**

In [5]:
water_path = '/content/drive/MyDrive/2023년 K-water 대국민 물 빅데이터 분석과제 공모전/data/다목적댐운영정보' # path 변수에 경로 지정

In [6]:
water_list = os.listdir(water_path)
print(water_list)
print(len(water_list))

['201901.csv', '201902.csv', '201903.csv', '201904.csv', '201905.csv', '201906.csv', '201907.csv', '201908.csv', '201909.csv', '201910.csv', '201912.csv', '202001.csv', '202002.csv', '202003.csv', '202004.csv', '202005.csv', '202006.csv', '202007.csv', '202008.csv', '202101.csv', '202102.csv', '202103.csv', '202104.csv', '202105.csv', '202106.csv', '202107.csv', '202108.csv', '202109.csv', '202110.csv', '202111.csv', '202112.csv', '202201.csv', '202204.csv', '202205.csv', '202206.csv', '202207.csv', '202209.csv', '202210.csv', '202211.csv', '202212.csv', '201911.csv', '202012.csv', '202202.csv', '202009.csv', '202010.csv', '202203.csv', '202011.csv', '202208.csv']
48


- 4개년치에 해당하는 48개 달에 대한 데이터가 정상적으로 load됨

In [7]:
df = file_concat(water_path, water_list)

In [8]:
df

,dam_nm,obsr_de,rain_qy,inflow_qy,tot_dcwtr_qy,rsvwt_qy,dam_rsvwt_rt
0,충주,20190101,0.0,35.381,104.637,2147.7650,78.10
1,횡성,20190101,0.0,0.373,1.614,71.9450,82.80
2,소양강,20190101,0.0,9.063,14.959,1864.7390,64.30
3,안동,20190101,0.0,1.122,16.978,929.9470,74.50
4,임하,20190101,0.0,0.000,8.555,374.2330,62.90
...,...,...,...,...,...,...,...
30676,부안,20220831,2.0,0.461,1.206,40.5367,80.59
30677,섬진강,20220831,6.0004,8.131,22.219,109.6671,23.53
30678,주암(본),20220831,1.1378,4.796,29.454,173.1312,37.88
30679,주암(조),20220831,0.7481,26.837,6.559,53.8232,21.53


- 365(일) * 4(년) * 21(댐 개수) = 30660
- 2020년의 경우 2월 29일까지 존재 -> 21개 추가

In [9]:
### 데이터 정보 확인

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30681 entries, 0 to 30680
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   dam_nm        30681 non-null  object 
 1   obsr_de       30681 non-null  int64  
 2   rain_qy       30681 non-null  object 
 3   inflow_qy     30681 non-null  float64
 4   tot_dcwtr_qy  30681 non-null  float64
 5   rsvwt_qy      30681 non-null  float64
 6   dam_rsvwt_rt  30681 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 1.6+ MB


- obsr_de(관측일)과 rain_qy(강우량)의 데이터형(dtype)이 잘못 설정되어 있는 것 같음
  - obsr_de: object -> datetime
  - rain_qy: object -> float64

In [10]:
# 결측치나 변경할 수 없는 값이 있다면 일단 모두 무시하고 데이터형 변경

df['obsr_de'] = df['obsr_de'].apply(lambda x: pd.to_datetime(x, format='%Y%m%d', errors = 'coerce'))
df['rain_qy'] = df['rain_qy'].apply(lambda x: pd.to_numeric(x, errors = 'coerce'))

In [11]:
df.head()

,dam_nm,obsr_de,rain_qy,inflow_qy,tot_dcwtr_qy,rsvwt_qy,dam_rsvwt_rt
0,충주,2019-01-01,0.0,35.381,104.637,2147.765,78.1
1,횡성,2019-01-01,0.0,0.373,1.614,71.945,82.8
2,소양강,2019-01-01,0.0,9.063,14.959,1864.739,64.3
3,안동,2019-01-01,0.0,1.122,16.978,929.947,74.5
4,임하,2019-01-01,0.0,0.000,8.555,374.233,62.9


In [12]:
df['dam_nm'].unique()

array(['충주', '횡성', '소양강', '안동', '임하', '성덕', '영주', '군위', '김천부항', '보현산',
       '합천', '남강', '밀양', '용담', '대청', '보령', '부안', '섬진강', '주암(본)', '주암(조)',
       '장흥'], dtype=object)

- 21개 댐에 대한 정보를 포함하고 있음

### **b) 이상치 처리**
- 저수량 데이터 특성상 값들 사이의 편차가 크다.
  - 이상치를 걸러내는 것에 의미가 없을 것으로 판단되어 별도로 진행 x

### **c) 결측치 처리**

In [13]:
# 현재 데이터 내의 결측치는 공백으로 처리되어 있음
# 데이터프레임 내의 공백을 NaN으로 변경

df.replace('', np.nan, inplace = True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30681 entries, 0 to 30680
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   dam_nm        30681 non-null  object        
 1   obsr_de       30681 non-null  datetime64[ns]
 2   rain_qy       30680 non-null  float64       
 3   inflow_qy     30681 non-null  float64       
 4   tot_dcwtr_qy  30681 non-null  float64       
 5   rsvwt_qy      30681 non-null  float64       
 6   dam_rsvwt_rt  30681 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 1.6+ MB


- rain_qy(강수량)에 결측치가 존재

In [15]:
df.loc[df['rain_qy'].isna(),:]

,dam_nm,obsr_de,rain_qy,inflow_qy,tot_dcwtr_qy,rsvwt_qy,dam_rsvwt_rt
659,김천부항,2019-02-01,NaN,0.026664,1.616854,37.143,68.4


In [16]:
# 해당 댐의 해당 연도 + 달의 강수량 최빈값으로 대체

df['rain_qy'] = df.groupby(['dam_nm', df['obsr_de'].dt.year ,df['obsr_de'].dt.month])['rain_qy'].transform(lambda x: x.fillna(x.mode().iloc[0]))

In [17]:
df.loc[df['rain_qy'].isna(),:]

,dam_nm,obsr_de,rain_qy,inflow_qy,tot_dcwtr_qy,rsvwt_qy,dam_rsvwt_rt


- 결측치가 제대로 채워짐

In [18]:
df.head()

,dam_nm,obsr_de,rain_qy,inflow_qy,tot_dcwtr_qy,rsvwt_qy,dam_rsvwt_rt
0,충주,2019-01-01,0.0,35.381,104.637,2147.765,78.1
1,횡성,2019-01-01,0.0,0.373,1.614,71.945,82.8
2,소양강,2019-01-01,0.0,9.063,14.959,1864.739,64.3
3,안동,2019-01-01,0.0,1.122,16.978,929.947,74.5
4,임하,2019-01-01,0.0,0.000,8.555,374.233,62.9


### **d) 단위 변경**
- 유입량, 방류량을 1일당 총량으로 변경
  - m3/s 단위를 m3로 변경
- 하루 기준: 60초 * 60분 * 24시간 = 86400초
- 총저수량의 경우 m3 * 백만을 m3으로 변경

In [19]:
df['inflow_qy'] = df['inflow_qy'] * 86400
df['tot_dcwtr_qy'] = df['tot_dcwtr_qy'] * 86400

In [20]:
df['rsvwt_qy'] = df['rsvwt_qy'] * 1000000

In [21]:
df.head()

,dam_nm,obsr_de,rain_qy,inflow_qy,tot_dcwtr_qy,rsvwt_qy,dam_rsvwt_rt
0,충주,2019-01-01,0.0,3056918.4,9040636.8,2.147765e+09,78.1
1,횡성,2019-01-01,0.0,32227.2,139449.6,7.194500e+07,82.8
2,소양강,2019-01-01,0.0,783043.2,1292457.6,1.864739e+09,64.3
3,안동,2019-01-01,0.0,96940.8,1466899.2,9.299470e+08,74.5
4,임하,2019-01-01,0.0,0.0,739152.0,3.742330e+08,62.9


In [22]:
duplicates = df[df.duplicated(subset = ['dam_nm', 'obsr_de'], keep = False)]
print(duplicates)

Empty DataFrame
Columns: [dam_nm, obsr_de, rain_qy, inflow_qy, tot_dcwtr_qy, rsvwt_qy, dam_rsvwt_rt]
Index: []


- 중복된 데이터는 x

## **1-2. 제원정보 데이터**
- 21개의 댐에 대한 제원 정보를 포함하는 데이터
  - 높이, 길이, 수위, 용량 등
- 해당 데이터에서 필요한 컬럼만을 선택하여 저장
  - 댐 이름, 유효저수량
- 활용 데이터 목록
  - [다목적댐 제원 정보](https://www.bigdata-environment.kr/user/data_market/detail.do?id=b74c6b60-38bb-11ea-be28-4fa0eb812a46)


### **📌Data Description**
1. dam_nm: 댐이름
2. valid_rsvwt_cpcty_qy: 유효저수용량

In [23]:
# 데이터 불러오기

df_valid_capa = pd.read_csv('/content/drive/MyDrive/2023년 K-water 대국민 물 빅데이터 분석과제 공모전/data/다목적댐_제원_정보.csv', sep = ',')
df_valid_capa

,dam_nm,valid_rsvwt_cpcty_qy
0,군위,40.10
1,김천부항,42.60
2,남강,299.70
3,대청,790.00
4,밀양,69.80
5,보령,108.70
6,보현산,17.88
7,부안,35.60
8,섬진강,429.00
9,성덕,24.80


In [24]:
df_valid_capa['dam_nm'].unique()

array(['군위', '김천부항', '남강', '대청', '밀양', '보령', '보현산', '부안', '섬진강', '성덕',
       '소양강', '안동', '영주', '용담', '임하', '장흥', '주암(본)', '주암(조)', '충주', '합천',
       '횡성'], dtype=object)

- 정상적으로 **21개**의 댐의 유효저수량이 포함되어 있음

In [25]:
# 두 데이터가 가진 댐이 서로 동일한지 확인

sum(~df['dam_nm'].isin(df_valid_capa['dam_nm']))

0

- 두 개의 데이터에서 동일한 댐에 대한 정보들을 가지고 있음을 확인할 수 있다.
  - 댐 이름에서 차이가 x

In [26]:
# 기존 df 와 df_valid_capa 병합하기(merge)

df = pd.merge(left = df, right = df_valid_capa,
              how = "inner", on = "dam_nm") # dam_nm 기준으로 결합

In [27]:
df

,dam_nm,obsr_de,rain_qy,inflow_qy,tot_dcwtr_qy,rsvwt_qy,dam_rsvwt_rt,valid_rsvwt_cpcty_qy
0,충주,2019-01-01,0.0,3056918.4,9040636.8,2.147765e+09,78.10,1789.0
1,충주,2019-01-02,0.0,2209852.8,9036748.8,2.140938e+09,77.90,1789.0
2,충주,2019-01-03,0.0,1373932.8,9039427.2,2.133272e+09,77.60,1789.0
3,충주,2019-01-04,0.0,2237846.4,9038476.8,2.126472e+09,77.30,1789.0
4,충주,2019-01-05,0.0,2248300.8,9036576.0,2.119683e+09,77.10,1789.0
...,...,...,...,...,...,...,...,...
30676,장흥,2022-08-27,0.0,16934.4,279331.2,7.934070e+07,41.54,171.0
30677,장흥,2022-08-28,0.0,88041.6,284515.2,7.914420e+07,41.44,171.0
30678,장흥,2022-08-29,0.0,82598.4,278899.2,7.894790e+07,41.33,171.0
30679,장흥,2022-08-30,0.0,25660.8,287020.8,7.868660e+07,41.20,171.0


- 정확히 결합된 것을 확인할 수 있음

## **1-3. 기상자료**
- 각 댐이 위치하는 지역의 기상정보에 대한 데이터
  - 습도, 기온, 풍속, 일사량
- 여러 데이터에서 필요한 정보들을 수집 후 최종 데이터 형태로 결합함
- 활용 데이터 목록
  - [환경 빅데이터 플랫폼_관측소별기상관측정보](https://www.bigdata-environment.kr/user/data_market/detail.do?id=b1de30f0-38bb-11ea-be28-4fa0eb812a46)
  - [국가수자원관리종합정보시스템_실시간 기상자료](http://www.wamis.go.kr/wkw/we_dwtwtobs.do)
  - [농촌진흥청_기상통계](https://fruit.nihhs.go.kr/main/aws/awsStat.do)
  - [국립농업과학원_농업기상정보_주산지기상분석](http://weather.rda.go.kr/w/farmProduce/frcPlpdAvg.do)
  - [농업관측통합시스템](https://oasis.krei.re.kr/basicInfo/etc/kma.do)

### **📌 Data Description**
1. dam_nm: 댐이름
2. anc_dt: 관측일자
3. avg_hmd: 평균습도
4. avg_tmr: 평균기온
5. avg_wv: 평균풍속
6. snsn_sum: 일사량

### **a) 데이터 불러오기**

In [28]:
weather_path = '/content/drive/MyDrive/2023년 K-water 대국민 물 빅데이터 분석과제 공모전/data/관측소별기상관측정보' # path 변수에 경로 지정

In [29]:
weather_list = os.listdir(weather_path)
print(weather_list)
print(len(weather_list))

['일사량 결측치 핸들링.pdf', '202002.csv', '202003.csv', '202004.csv', '202005.csv', '202006.csv', '202008.csv', '202009.csv', '202010.csv', '202011.csv', '202101.csv', '202102.csv', '202103.csv', '202104.csv', '202105.csv', '202106.csv', '202207.csv', '202001.csv', '202007.csv', '202012.csv', '202107.csv', '202112.csv', '202202.csv', '202203.csv', '202204.csv', '202205.csv', '202206.csv', '202209.csv', '201901_201906.csv', '202108.csv', '202109.csv', '202208.csv', '202201.csv', '202210.csv', '202212.csv', '202211.csv', '201907_201912.csv', '202110.csv', '202111.csv']
39


In [30]:
df2 = file_concat(weather_path, weather_list)

In [31]:
df2

,dam_nm,anc_dt,avg_hmd,avg_tmr,avg_wv,snsn_sum
0,군위,20200201,67.6,2.9,2.0,7.1
1,군위,20200202,61.5,3.6,1.2,8.3
2,군위,20200203,45.6,3.1,2.9,7.2
3,군위,20200204,42.5,1.9,1.8,9.8
4,군위,20200205,30.8,-1.5,4.5,10.1
...,...,...,...,...,...,...
30676,횡성,20211126,67.6,0.8,0.8,4.4
30677,횡성,20211127,68.6,-0.2,0.4,6.1
30678,횡성,20211128,63.1,1.1,0.3,6.1
30679,횡성,20211129,71.8,2.6,0.3,4.2


In [32]:
### 데이터 정보 확인

df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30681 entries, 0 to 30680
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   dam_nm    30681 non-null  object 
 1   anc_dt    30681 non-null  int64  
 2   avg_hmd   30620 non-null  float64
 3   avg_tmr   30676 non-null  float64
 4   avg_wv    30674 non-null  float64
 5   snsn_sum  30664 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 1.4+ MB


- 변수명 통일
  - 관측일자의 경우 다른 데이터와의 병합을 위해 ```anc_dt```에서 ```obsr_de```로 변경
- 데이터형 변경
  - obsr_de: object -> datetime
- 결측치 처리 필요

In [33]:
# 변수명 변경

df2.rename(columns={'anc_dt':'obsr_de'}, inplace = True)

In [34]:
df2

,dam_nm,obsr_de,avg_hmd,avg_tmr,avg_wv,snsn_sum
0,군위,20200201,67.6,2.9,2.0,7.1
1,군위,20200202,61.5,3.6,1.2,8.3
2,군위,20200203,45.6,3.1,2.9,7.2
3,군위,20200204,42.5,1.9,1.8,9.8
4,군위,20200205,30.8,-1.5,4.5,10.1
...,...,...,...,...,...,...
30676,횡성,20211126,67.6,0.8,0.8,4.4
30677,횡성,20211127,68.6,-0.2,0.4,6.1
30678,횡성,20211128,63.1,1.1,0.3,6.1
30679,횡성,20211129,71.8,2.6,0.3,4.2


In [35]:
# 결측치나 변경할 수 없는 값이 있다면 일단 모두 무시하고 데이터형 변경

df2['obsr_de'] = df2['obsr_de'].apply(lambda x: pd.to_datetime(x, format = '%Y%m%d',errors = 'coerce'))

### **b) 이상치 처리**
- 기상 데이터에 대해서만 이상치 처리  
(일단은 수행 x)

In [36]:
### 이상치 탐지 함수

def get_outlier(df = None, dam_name = None, year = None, month = None, column = None, weight = 1.5):
    # 1/4 분위와 3/4 분위 지점을 np.percentile로 구함
    df_col = df[column]
    quantile_25 = np.percentile(df_col.values, 25)
    quantile_75 = np.percentile(df_col.values, 75)

    # IQR을 구하고, IQR에 1.5를 곱하여 최대값과 최소값 지점 구함.
    iqr = quantile_75 - quantile_25
    iqr_weight = iqr * weight
    lowest_val = quantile_25 - iqr_weight
    highest_val = quantile_75 + iqr_weight

    # 최대값 보다 크거나, 최소값 보다 작은 값을 이상치로 설정하고 DataFrame index 반환
    outlier_index = df_col[(df_col < lowest_val) | (df_col > highest_val)].index

    # 이상치 정보를 저장할 dictionary 생성
    outlier_info = {
        'dam_nm': dam_name,
        'obsr_de': f'{year}년 {month}월',
        'column': column,
        'outlier_idx': outlier_index
    }

    return outlier_info

In [37]:
# 댐 이름, 월, 컬럼명을 저장할 리스트 생성
outlier_info_list = []

cols = ['avg_hmd', 'avg_tmr', 'avg_wv', 'snsn_sum']
grouped = df2.groupby(['dam_nm', df['obsr_de'].dt.year, df['obsr_de'].dt.month])  # 댐별로 + 연도별 + 월별로

for col in cols:
  for (dam, year, month), group_data in grouped:
    outlier_info = get_outlier(group_data, dam_name=dam, year=year, month=month, column=col, weight=1.5)
    if len(outlier_info['outlier_idx']) > 0:
      outlier_info_list.append(outlier_info)

# 이상치 정보 출력
for outlier_info in outlier_info_list:
  dam_name = outlier_info['dam_nm']
  obsr_de = outlier_info['obsr_de']
  col = outlier_info['column']
  outlier_idx = outlier_info['outlier_idx']
  print('댐: {}, 월: {}, 컬럼: {}, 이상치 데이터 인덱스: {}'.format(dam_name, obsr_de, col, outlier_idx))
  print()

  # 이상치가 포함된 행 삭제
  # df.drop(outlier_idx, inplace = True) -- 일단은 수행 안함

댐: 군위, 월: 2019년 6월, 컬럼: avg_hmd, 이상치 데이터 인덱스: Int64Index([8923], dtype='int64')

댐: 군위, 월: 2019년 7월, 컬럼: avg_hmd, 이상치 데이터 인덱스: Int64Index([29416], dtype='int64')

댐: 군위, 월: 2019년 9월, 컬럼: avg_hmd, 이상치 데이터 인덱스: Int64Index([3177, 3178, 3180], dtype='int64')

댐: 군위, 월: 2020년 2월, 컬럼: avg_hmd, 이상치 데이터 인덱스: Int64Index([22303], dtype='int64')

댐: 군위, 월: 2020년 5월, 컬럼: avg_hmd, 이상치 데이터 인덱스: Int64Index([1917, 12144, 12146], dtype='int64')

댐: 군위, 월: 2020년 9월, 컬럼: avg_hmd, 이상치 데이터 인덱스: Int64Index([17399], dtype='int64')

댐: 군위, 월: 2020년 11월, 컬럼: avg_hmd, 이상치 데이터 인덱스: Int64Index([10190, 10191], dtype='int64')

댐: 군위, 월: 2020년 12월, 컬럼: avg_hmd, 이상치 데이터 인덱스: Int64Index([1276, 1278, 17335], dtype='int64')

댐: 군위, 월: 2021년 6월, 컬럼: avg_hmd, 이상치 데이터 인덱스: Int64Index([5130, 25593, 25594, 25595], dtype='int64')

댐: 군위, 월: 2021년 9월, 컬럼: avg_hmd, 이상치 데이터 인덱스: Int64Index([25672, 25675], dtype='int64')

댐: 군위, 월: 2021년 11월, 컬럼: avg_hmd, 이상치 데이터 인덱스: Int64Index([14032, 14033, 24259], dtype='int64')

댐: 군위, 월: 20

In [38]:
# 이상치 처리 후 DataFrame
# df

### **c) 결측치 처리**

In [39]:
# 현재 데이터 내의 결측치는 공백으로 처리되어 있음
# 데이터프레임 내의 공백을 NaN으로 변경

df2.replace('', np.nan, inplace = True)

In [40]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30681 entries, 0 to 30680
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   dam_nm    30681 non-null  object        
 1   obsr_de   30681 non-null  datetime64[ns]
 2   avg_hmd   30620 non-null  float64       
 3   avg_tmr   30676 non-null  float64       
 4   avg_wv    30674 non-null  float64       
 5   snsn_sum  30664 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 1.4+ MB


- 여러 변수들에 걸쳐 결측치가 있는 것으로 보인다.
- 결측치 처리
  - 습도 -> 최빈값
  - 온도 -> 평균
  - 풍속 -> 평균
  - 일사량 -> 최빈값

In [41]:
## 습도
len(df2.loc[df2['avg_hmd'].isna(),:])

61

In [42]:
# 해당 댐의 해당 연도 + 달의 습도 최빈값으로 대체

df2['avg_hmd'] = df2.groupby(['dam_nm',df2['obsr_de'].dt.year, df2['obsr_de'].dt.month])['avg_hmd'].transform(lambda x: x.fillna(x.mode().iloc[0]))

In [43]:
df2.loc[df2['avg_hmd'].isna(),:]

,dam_nm,obsr_de,avg_hmd,avg_tmr,avg_wv,snsn_sum


In [44]:
## 온도

len(df2.loc[df2['avg_tmr'].isna(),:])

5

In [45]:
# 해당 댐의 해당 연도 + 달의 온도 평균값으로 대체

df2['avg_tmr'] = df2.groupby(['dam_nm',df2['obsr_de'].dt.year, df2['obsr_de'].dt.month])['avg_tmr'].transform(lambda x: x.fillna(x.mean()))

In [46]:
df2.loc[df2['avg_tmr'].isna(),:]

,dam_nm,obsr_de,avg_hmd,avg_tmr,avg_wv,snsn_sum


In [47]:
## 풍속

len(df2.loc[df2['avg_wv'].isna(),:])

7

In [48]:
# 해당 댐의 해당 연도+달의 온도 평균값으로 대체

df2['avg_wv'] = df2.groupby(['dam_nm',df2['obsr_de'].dt.year, df2['obsr_de'].dt.month])['avg_wv'].transform(lambda x: x.fillna(x.mean()))

In [49]:
df2.loc[df2['avg_wv'].isna(),:]

,dam_nm,obsr_de,avg_hmd,avg_tmr,avg_wv,snsn_sum


In [50]:
## 일사량

len(df2.loc[df2['snsn_sum'].isna(),:])

17

In [51]:
# 해당 댐의 해당 연도 + 달의 습도 최빈값으로 대체

df2['snsn_sum'] = df2.groupby(['dam_nm',df2['obsr_de'].dt.year, df2['obsr_de'].dt.month])['snsn_sum'].transform(lambda x: x.fillna(x.mode().iloc[0]))

In [52]:
df2.loc[df2['snsn_sum'].isna(),:]

,dam_nm,obsr_de,avg_hmd,avg_tmr,avg_wv,snsn_sum


In [53]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30681 entries, 0 to 30680
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   dam_nm    30681 non-null  object        
 1   obsr_de   30681 non-null  datetime64[ns]
 2   avg_hmd   30681 non-null  float64       
 3   avg_tmr   30681 non-null  float64       
 4   avg_wv    30681 non-null  float64       
 5   snsn_sum  30681 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 1.4+ MB


- 결측치가 제대로 채워졌다.

In [54]:
sum(~df['dam_nm'].isin(df2['dam_nm']))

0

- 댐 이름은 모두 동일

In [55]:
sum(~df['obsr_de'].isin(df2['obsr_de']))

0

- 관측일자도 모두 동일

In [56]:
duplicates2 = df2[df2.duplicated(subset = ['dam_nm', 'obsr_de'], keep=False)]
print(duplicates2)

Empty DataFrame
Columns: [dam_nm, obsr_de, avg_hmd, avg_tmr, avg_wv, snsn_sum]
Index: []


- 중복된 데이터는 없음을 확인할 수 있다.

In [57]:
### 데이터 병합

df = pd.merge(left = df, right = df2, how = "inner", on = ['dam_nm','obsr_de'] ) # dam_nm, 관측일 기준으로 조인

In [58]:
df

,dam_nm,obsr_de,rain_qy,inflow_qy,tot_dcwtr_qy,rsvwt_qy,dam_rsvwt_rt,valid_rsvwt_cpcty_qy,avg_hmd,avg_tmr,avg_wv,snsn_sum
0,충주,2019-01-01,0.0,3056918.4,9040636.8,2.147765e+09,78.10,1789.0,59.7,-4.2,1.7,5.8
1,충주,2019-01-02,0.0,2209852.8,9036748.8,2.140938e+09,77.90,1789.0,57.6,-6.4,1.0,10.1
2,충주,2019-01-03,0.0,1373932.8,9039427.2,2.133272e+09,77.60,1789.0,54.5,-6.3,0.8,10.1
3,충주,2019-01-04,0.0,2237846.4,9038476.8,2.126472e+09,77.30,1789.0,57.7,-4.2,0.6,6.7
4,충주,2019-01-05,0.0,2248300.8,9036576.0,2.119683e+09,77.10,1789.0,52.7,-2.4,1.1,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...
30676,장흥,2022-08-27,0.0,16934.4,279331.2,7.934070e+07,41.54,171.0,75.6,22.0,1.7,9.2
30677,장흥,2022-08-28,0.0,88041.6,284515.2,7.914420e+07,41.44,171.0,79.4,20.2,1.0,3.3
30678,장흥,2022-08-29,0.0,82598.4,278899.2,7.894790e+07,41.33,171.0,80.3,22.5,0.7,1.6
30679,장흥,2022-08-30,0.0,25660.8,287020.8,7.868660e+07,41.20,171.0,94.6,23.9,0.9,7.8


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30681 entries, 0 to 30680
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   dam_nm                30681 non-null  object        
 1   obsr_de               30681 non-null  datetime64[ns]
 2   rain_qy               30681 non-null  float64       
 3   inflow_qy             30681 non-null  float64       
 4   tot_dcwtr_qy          30681 non-null  float64       
 5   rsvwt_qy              30681 non-null  float64       
 6   dam_rsvwt_rt          30681 non-null  float64       
 7   valid_rsvwt_cpcty_qy  30681 non-null  float64       
 8   avg_hmd               30681 non-null  float64       
 9   avg_tmr               30681 non-null  float64       
 10  avg_wv                30681 non-null  float64       
 11  snsn_sum              30681 non-null  float64       
dtypes: datetime64[ns](1), float64(10), object(1)
memory usage: 3.0+ MB


- 결측치 없이 잘 처리되었다.

In [60]:
df.to_csv("/content/drive/MyDrive/2023년 K-water 대국민 물 빅데이터 분석과제 공모전/data/병합 데이터(수빈)/final.csv")